In [3]:
import pandas as pd
import boto3
import re 
import os
import pandas as pd
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin
import random
import pandas as pd
import string
import  json
from tqdm import tqdm 
from sklearn.model_selection import train_test_split


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/cuda/__init__.py:616: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [6]:
dataframe = pd.read_excel("tempexcep.xlsx")

def get_start_end_re(data,word):
    if re.search(r'\b'+re.escape(word)+r'\b', data):
        return data.find(word),data.find(word)+len(word)-1
    else:
        return False
    
    
def createDateSpacyFormat(text_data,label,entityName):
    spacyFormat = []
    for count in tqdm(range(len(text_data))):
        tupList = []
        entDict = {}
#         tupList.append(text_data[count])
        if get_start_end_re(text_data[count],label[count]) == False:
            continue
        else:
            start_index, end_index = get_start_end_re(text_data[count],label[count])
            temp_list = list(('start_index', 'end_index', 'Word'))
            temp_list[0] = start_index
            temp_list[1] = end_index
            temp_list[2] = entityName
            entList = []
            entList.append(tuple(temp_list))
            entDict['entities'] = entList
            tupList.append(text_data[count])
            tupList.append(entDict)
            spacyFormat.append(tuple(tupList))
    return spacyFormat

data1 = list(dataframe['PageText'])
data2 = list(dataframe['compDate'])
X_train, X_test, y_train, y_test = train_test_split(data1,data2,test_size=0.2)


nlp = spacy.blank("en") # load a new spacy model
def create_training(TRAIN_DATA):
    
    db = DocBin() # create a DocBin object

    for text, annot in tqdm(TRAIN_DATA): # data in previous format
        doc = nlp.make_doc(text) # create doc object from text
        ents = []
#         print(text)
#         print(annot[])
        for start, end, label in annot["entities"]: # add character indexes
            span = doc.char_span(start, end, label=label, alignment_mode="expand")
            print("*********** Starting*****************")
            print(span)
            print(start)
            print(end)
            print(label)
            print("*********** Ending*****************")
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents # label the text with the ents
        db.add(doc)
    return (db)
X_train, X_test, y_train, y_test = train_test_split(data1,data2,test_size=0.2)
spacyformat = createDateSpacyFormat(X_train,y_train,"IncepDate")
X_tr, X_te= train_test_split(spacyformat,test_size=0.2)

new_training_data = create_training(X_tr[:100])
new_training_data.to_disk("train.spacy")

new_testing_data = create_training(X_te[:50])
new_testing_data.to_disk("dev.spacy")


os.system('! python -m spacy init fill-config base_config.cfg config.cfg')
os.system('python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy')



 70%|███████   | 70/100 [00:00<00:00, 354.98it/s]

*********** Starting*****************
1st july 2020
297
309
IncepDate
*********** Ending*****************
*********** Starting*****************
01 july 2021
321
332
IncepDate
*********** Ending*****************
*********** Starting*****************
05 july 2020
496
507
IncepDate
*********** Ending*****************
*********** Starting*****************
01 february 2020
322
337
IncepDate
*********** Ending*****************
*********** Starting*****************
1st september 2021
1103
1120
IncepDate
*********** Ending*****************
*********** Starting*****************
1 july 2021
498
508
IncepDate
*********** Ending*****************
*********** Starting*****************
1st december 2020
265
281
IncepDate
*********** Ending*****************
*********** Starting*****************
01 december 2020
510
525
IncepDate
*********** Ending*****************
*********** Starting*****************
30th september 2021
115
133
IncepDate
*********** Ending*****************
*********** Starting*******

100%|██████████| 50/50 [00:00<00:00, 379.16it/s]


*********** Starting*****************
01 july 2020
492
503
IncepDate
*********** Ending*****************
*********** Starting*****************
29th june 2020
1088
1101
IncepDate
*********** Ending*****************
*********** Starting*****************
4th september 2021
243
260
IncepDate
*********** Ending*****************
*********** Starting*****************
30 june 2020
191
202
IncepDate
*********** Ending*****************
*********** Starting*****************
19 october 2021
454
468
IncepDate
*********** Ending*****************
*********** Starting*****************
26th april 2020
136
150
IncepDate
*********** Ending*****************
*********** Starting*****************
31st october, 2020
521
538
IncepDate
*********** Ending*****************
*********** Starting*****************
1 january 2021
381
394
IncepDate
*********** Ending*****************
*********** Starting*****************
1st september 2021
97
114
IncepDate
*********** Ending*****************
*********** Starting******

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/cuda/__init__.py:616: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/cuda/__init__.py:616: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     95.17    0.00    0.00    0.00    0.00
  2     200        715.12   2194.81   96.97   97.96   96.00    0.97
  4     400         45.21     38.41   95.15   92.45   98.00    0.95
  6     600         11.94     18.03   94.95   95.92   94.00    0.95
  8     800          7.78      8.96   93.75   97.83   90.00    0.94
 10    1000         17.04     16.31   95.05   94.12   96.00    0.95
 12    1200         29.86     16.53   95.92   97.92   94.00    0.96
 14    1400          6.95      4.92   95.92   97.92   94.00    0

0

In [7]:
nlp = spacy.load('output/model-best')

In [ ]:
filename = 'spacy_model.p'
pickle.dump(model, open(filename, 'wb'))